# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!

  - 🤝 Breakout Room #2:
  1. Evaluating the LangGraph Application with LangSmith
  2. Adding Helpfulness Check and "Loop" Limits
  3. LangGraph for the "Patterns" of GenAI

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effectively allowing us to recreate application flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies

We'll first install all our required libraries.

> NOTE: If you're running this locally - please skip this step.

In [1]:
!pip install -qU langchain langchain_openai langchain-community langgraph arxiv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.4/412.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.1 MB/s eta 0:00:00


## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [2]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE5 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Tavily Search Results](https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/tools/tavily_search/tool.py)
- [Arxiv](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/arxiv)

####🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

#### ✅ Answer
1. Tavily
2. arXiv

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tavily_tool = TavilySearchResults(max_results=5)

tool_belt = [
    tavily_tool,
    ArxivQueryRun(),
]

### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [6]:
model = model.bind_tools(tool_belt)

#### ❓ Question #1:

How does the model determine which tool to use?

#### Answer #1: 
It determines selection of tools dynamically during inference, some of the selection creteria are following 
1. User Query Matching: If the user's query explicitly mentions research papers, scientific topics, or academic sources, the model is more likely to select ArxivQueryRun().
If the query is more general (e.g., web search or current events), it might favor tavily_tool.
2. Tool Descriptions (Metadata):
Each tool usually has a description that helps the model understand its purpose. If ArxivQueryRun() is described as "a tool to fetch academic papers from arXiv," the model will use it when research papers are relevant.
Few-shot Prompting & Examples:
3. If LangChain is given examples where ArxivQueryRun() is used for academic queries and tavily_tool for general queries, it learns to choose correctly based on context.
Priority & Overlapping Functionality:

If both tools can handle a query, the model might choose based on priority (implicit in LangChain’s decision logic) or return results from both if it finds ambiguity.

## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [7]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [8]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [9]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [10]:
uncompiled_graph.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [11]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [12]:
uncompiled_graph.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [13]:
compiled_graph = uncompiled_graph.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?

If not, how could we impose a limit to the number of cycles?

#### Answers: #2:
LangGraph does not have built-in cycle limits, but you can enforce them by:
1. Tracking cycles in state (best for custom logic)
   -  CycleState keeps track of the number of cycles, increment_counter() updates the count, check_limit()
2. Using max_loops in RunnableConfig (simpler, global enforcement)

## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [14]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Who is the current captain of the Winnipeg Jets?")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JYK4qQhjKzv792qnqwXBOYBU', 'function': {'arguments': '{"query":"current captain of the Winnipeg Jets 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 162, 'total_tokens': 189, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e490e116-b38c-471f-a885-110138fa73d5-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current captain of the Winnipeg Jets 2023'}, 'id': 'call_JYK4qQhjKzv792qnqwXBOYBU', 'type': 'tool_call'}], usage_metadata={'input_tokens': 162, 'output_t

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [15]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the QLoRA paper, then search each of the authors to find out their latest Tweet using Tavily!")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fe6LfYqRxhFvwpCDNf9jgtzT', 'function': {'arguments': '{"query":"QLoRA"}', 'name': 'arxiv'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 178, 'total_tokens': 195, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-42ab7859-14e4-45de-b1d3-8c9fb7f18f03-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'QLoRA'}, 'id': 'call_fe6LfYqRxhFvwpCDNf9jgtzT', 'type': 'tool_call'}], usage_metadata={'input_tokens': 178, 'output_tokens': 17, 'total_tokens': 195, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'a

####🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.


#### ✅ Answer:

From the given payload, we can break down the agent's steps in reaching the final answer:  

### **Step-by-Step Analysis**  

1. **Agent Calls Arxiv for QLoRA**  
   - The agent detects that it needs information on **QLoRA**.  
   - It makes a **tool call to Arxiv** (`call_fe6LfYqRxhFvwpCDNf9jgtzT`).  

2. **Action Node Uses Arxiv**  
   - The `action` node executes the **Arxiv search** and retrieves **three papers** on QLoRA.  

3. **Agent Calls Tavily for Twitter Information**  
   - The agent then looks for **latest tweets** from:
     - **Tim Dettmers**
     - **Artidoro Pagnoni**
     - **Ari Holtzman**
     - **Luke Zettlemoyer**  
   - It makes **four tool calls** to `tavily_search_results_json` (for each person).  

4. **Action Node Fails Due to Unauthorized Error (401)**  
   - The `action` node attempts to fetch Twitter data but encounters **authorization errors** (`Error 401: Unauthorized`).  

5. **Agent Recognizes Failure & Responds Accordingly**  
   - Since Tavily search **failed**, the agent **informs the user** that it couldn't retrieve the tweets and advises checking the profiles manually.  

### **Total Steps Taken**  
1. **Call Arxiv for QLoRA** ✅  
2. **Process Arxiv Response** ✅  
3. **Call Tavily for Tweets (4 tool calls)** 🚫 (Fails)  
4. **Process Tavily Failure & Generate Final Answer** ✅  

Thus, the agent took **5 steps** in total:
- **3 successful steps**
- **1 failed batch of 4 calls**
- **1 final step to inform the user**  

## Part 1: LangSmith Evaluator

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [16]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain = convert_inputs | compiled_graph | parse_output

In [17]:
agent_chain.invoke({"question" : "What is RAG?"})

'RAG stands for Retrieval-Augmented Generation. It is a technique used in natural language processing (NLP) that combines retrieval-based methods with generative models to improve the quality and relevance of generated text. The main idea behind RAG is to enhance the capabilities of generative models by incorporating external information retrieved from a large corpus of documents or a database.\n\nIn a typical RAG setup, when a query is presented, the system first retrieves relevant documents or passages from a pre-existing dataset. These retrieved documents are then used as additional context or input for a generative model, such as a transformer-based language model, to produce a more informed and contextually accurate response.\n\nRAG is particularly useful in scenarios where the generative model alone might not have enough information to generate a high-quality response, such as in open-domain question answering or dialogue systems. By leveraging both retrieval and generation, RAG 

### Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]
```

####🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions.

#### ✅  Answer
- Based on the formate created questions and answer Dataset

In [18]:
questions = [
    "What is the main advantage of quantized models?",
    "What does LoRA stand for in the context of AI models?",
    "What is a key benefit of using Low-Rank Adaptation?",
    "Who introduced the concept of quantization in AI models?",
    "What does 'RLHF' stand for in AI training methods?"
]

answers = [
    {"must_mention": ["storage", "compute"]},
    {"must_mention": ["Low-Rank", "Adaptation"]},
    {"must_mention": ["efficient", "training"]},
    {"must_mention": ["Hinton", "Geoffrey"]},
    {"must_mention": ["Reinforcement", "Feedback"]},
]


Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [19]:
from langsmith import Client

client = Client()

dataset_name = f"Retrieval Augmented Generation - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    outputs=answers,
    dataset_id=dataset.id,
)

#### ❓ Question #3:

How are the correct answers associated with the questions?

> NOTE: Feel free to indicate if this is problematic or not

#### $ Answer #3:
- The first question in questions corresponds to the first dictionary in answers.
- The second question corresponds to the second dictionary, and so on.
- The first question (Index 0) expects an answer containing "paged" and "optimizer".
- The second question (Index 1) expects "NF4" and "NormalFloat".

Pros of This Approach:
1. Simple & Easy to Implement – Just use indexing to match questions and answers.
2. Lightweight – No need for complex mappings or additional structures.
3. Works Well for Small Lists – If the number of questions is small and manually curated, this approach is manageable.

Potential Issues:
1. Index Mismatch Risk – If someone modifies one list but forgets to update the other, the questions and answers will be misaligned.
2. Scalability Issues – If the dataset grows significantly, maintaining two separate lists becomes harder.
3. Harder to Debug – If a mismatch happens, debugging can be tedious since errors won’t be immediately obvious.


### Task 2: Adding Evaluators

Now we can add a custom evaluator to see if our responses contain the expected information.

We'll be using a fairly naive exact-match process to determine if our response contains specific strings.

In [20]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_mention(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_mention", score=score)

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

#### $ Answer #4: 

##### Strengths of the Current Approach
    1. Simple & Efficient – Checks if all required phrases exist in the prediction.
    2. Boolean Scoring (0 or 1) – Easy to interpret results.
    3. Leverages run_evaluator – Works well with LangSmith evaluation pipelines.

##### Gaps & Potential Issues
    1. Exact Match Requirement is Too Strict
    The current check requires an exact substring match. If a phrase appears in a slightly different form (e.g., pluralized or reworded), it fails unfairly.
    2. No Consideration for Case Sensitivity
    "NormalFloat" and "normalfloat" are considered different, which may lead to unfair scoring.
    3. Doesn’t Handle Word Boundaries Well
    4. Binary Scoring Ignores Partial Credit
    If a response contains some but not all required phrases, it gets a 0 score, even if it's mostly correct.
    5. No Explanation for Failures
    Currently, if the evaluator fails, it doesn’t tell why (which phrases were missing).

Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [21]:
experiment_results = client.evaluate(
    agent_chain,
    data=dataset_name,
    evaluators=[must_mention],
    experiment_prefix=f"RAG Pipeline - Evaluation - {uuid4().hex[0:4]}",
    metadata={"version": "1.0.0"},
)

View the evaluation results for experiment: 'RAG Pipeline - Evaluation - a002-97a9d36b' at:
https://smith.langchain.com/o/8a88797e-5f0c-4143-8820-5e498815cee3/datasets/5ad04ad1-2976-4161-9f14-e1ced0765166/compare?selectedSessions=b9f6b7dd-25e7-408a-b2cc-4b53c8367e61




0it [00:00, ?it/s]

In [22]:
print(experiment_results)

<ExperimentResults RAG Pipeline - Evaluation - a002-97a9d36b>


## Part 2: LangGraph with Helpfulness:

### Task 3: Adding Helpfulness Check and "Loop" Limits

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add to our existing conditional edge to obtain the behaviour we desire.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [23]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

####🏗️ Activity #5:

Please write markdown for the following cells to explain what each is doing.

##### ✅ Answer
1. ***`graph_with_helpfulness_check = StateGraph(AgentState)`***  
   - Creates a **state graph** named `graph_with_helpfulness_check`.  
   - `StateGraph(AgentState)` suggests that this graph tracks different states of an agent.  

2. ***`graph_with_helpfulness_check.add_node("agent", call_model)`***
   - Adds a **node** labeled `"agent"` to the graph.  
   - This node is associated with the function `call_model`, meaning:
     - When the state is `"agent"`, the `call_model` function is executed. 

3. ***`graph_with_helpfulness_check.add_node("action", tool_node)`***
   - Adds another **node** labeled `"action"`, associated with `tool_node`.  

In [24]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", tool_node)

##### ✅ Answer
1. ***`set_entry_point("agent")`*** - Setting an entry point for the node Agent.

In [25]:
graph_with_helpfulness_check.set_entry_point("agent")

In [27]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def tool_call_or_helpful(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  if len(state["messages"]) > 10:
    return "END"

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4")

  helpfulness_chain = prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    return "end"
  else:
    return "continue"

####🏗️ Activity #4:

Please write what is happening in our `tool_call_or_helpful` function!

#### ✅ Answer #4:

##### **What is Happening in ****`tool_call_or_helpful`****?**

The function **determines the next step in a conversation** based on message history.  

### **Step-by-Step Breakdown**

1. **Check if the Last Message Contains a Tool Call**

   - If `last_message.tool_calls` exists, return `"action"` (indicating that a tool needs to be called).

2. **Retrieve the Initial Query & Final Response**

   - Extracts `initial_query` (first message) and `final_response` (last message).

3. **Stop if Message Count Exceeds 10**

   - If the number of messages is **greater than 10**, return `"END"` to prevent infinite loops.

4. **Evaluate the Helpfulness of the Response**

   - Uses a **GPT-4 model** to determine if the **final response is highly useful**:
     - A prompt is created, inserting the `initial_query` and `final_response`.
     - This prompt is passed to the `helpfulness_check_model`.
     - The output is parsed to extract `"Y"` (helpful) or `"N"` (not helpful).

5. **Decide the Next Action Based on Helpfulness**

   - If the response contains `"Y"`, return `"end"` (stop further conversation).
   - Otherwise, return `"continue"` (indicating more interaction is needed).



### **Summary of Decisions Made**

| Condition                     | Output       |
| ----------------------------- | ------------ |
| Last message has tool calls   | `"action"`   |
| More than 10 messages         | `"END"`      |
| Response is helpful (`"Y"`)   | `"end"`      |
| Response is not helpful (`"N"`) | `"continue"` |


This function helps control conversation flow, ensuring it calls tools when needed, limits message cycles, and ends when a useful response is reached.

##### ✅ Answer
1. **Starting from `"agent"`** → The function `tool_call_or_helpful` is called.  
2. **Based on its return value, the next state is chosen**:  
   - `"continue"` → Stay in `"agent"` (keeps generating responses).  
   - `"action"` → Move to `"action"` (execute a tool call).  
   - `"end"` → Transition to `END` (conversation stops).  

In [28]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    tool_call_or_helpful,
    {
        "continue" : "agent",
        "action" : "action",
        "end" : END
    }
)

##### ✅ Answer
Final step to add eddge between Agent and action

In [29]:
graph_with_helpfulness_check.add_edge("action", "agent")

##### ✅ Answer

1. **`graph_with_helpfulness_check.compile()`**  
   - Converts the defined state graph into an **executable agent**.  
   - It ensures all nodes, edges, and logic are properly set up.  

2. **Assigns the compiled agent to `agent_with_helpfulness_check`**  
   - This object can now be used to process conversations.  
   - It follows the **state transitions** based on `tool_call_or_helpful`.  



In [30]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

##### ✅ Answer

1. **`inputs = {"messages": [...]}`**  
   - Defines an input message where a human asks multiple machine learning-related questions.

2. **`async for chunk in agent_with_helpfulness_check.astream(...)`**  
   - Streams responses **as they are generated** by the compiled agent.  
   - `astream()` processes the conversation **incrementally** rather than waiting for the full response.

3. **`stream_mode="updates"`**  
   - Indicates that **only updated parts of the conversation** should be streamed, rather than re-sending the full history each time.

4. **Processing Each Streamed Update:**
   - Loops through `chunk.items()`, where each `node` (state in the graph) sends updates.
   - `print(f"Receiving update from node: '{node}'")` → Identifies which state is producing updates.
   - `print(values["messages"])` → Displays the response messages from that state.

In [31]:
inputs = {"messages" : [HumanMessage(content="Related to machine learning, what is LoRA? Also, who is Tim Dettmers? Also, what is Attention?")]}

async for chunk in agent_with_helpfulness_check.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_P7omLB81EYvAO5xPWP75wVB4', 'function': {'arguments': '{"query": "LoRA in machine learning"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_IwmzJX5ulLCUWEOzE1CRhqWF', 'function': {'arguments': '{"query": "Tim Dettmers"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_6Z46xOFGO50vouyYMzeWYXN6', 'function': {'arguments': '{"query": "Attention in machine learning"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 177, 'total_tokens': 258, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_rea

### Task 4: LangGraph for the "Patterns" of GenAI

Let's ask our system about the 4 patterns of Generative AI:

1. Prompt Engineering
2. RAG
3. Fine-tuning
4. Agents

In [32]:
patterns = ["prompt engineering", "RAG", "fine-tuning", "LLM-based agents"]

In [33]:
for pattern in patterns:
  what_is_string = f"What is {pattern} and when did it break onto the scene??"
  inputs = {"messages" : [HumanMessage(content=what_is_string)]}
  messages = agent_with_helpfulness_check.invoke(inputs)
  print(messages["messages"][-1].content)
  print("\n\n")

It seems there was an issue accessing the search engine for the latest information. However, I can provide a general overview based on my existing knowledge.

**Prompt Engineering** is a concept in the field of artificial intelligence, particularly in natural language processing (NLP). It involves designing and refining prompts given to language models to elicit the most accurate or desired responses. This process can include crafting specific questions, statements, or instructions that guide the model to produce useful outputs.

Prompt engineering became more prominent with the rise of large language models like GPT-3, which demonstrated the ability to perform a wide range of tasks based on the prompts they receive. The concept gained traction as researchers and developers realized that the way a question or task is presented to a model can significantly impact the quality and relevance of the model's response.

The term "prompt engineering" started gaining popularity around 2020, coi